In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [2]:

# --- Configuration (Adjust these paths and model names) ---
# Replace with the actual path to your technical book (e.g., "C:/Users/YourUser/Documents/my_tech_book.pdf")
BOOK_PATH = "Encyclopedia of Machine Learning.pdf" 
# Choose one of the small LLMs you pulled with Ollama (e.g., "gemma:2b", "phi3", "qwen:3b", "mistral")
OLLAMA_MODEL_NAME = "gemma:2b" 
# Embedding model (all-MiniLM-L6-v2 is a good default, or "nomic-embed-text" if using Ollama for embeddings)
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2" # "all-MiniLM-L6-v2"
# Path to save/load your FAISS vector store to avoid re-embedding
FAISS_INDEX_PATH = "faiss_index_tech_book"


In [3]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")
# # In case you want to reduce the maximum sequence length:
# model.max_seq_length = 4096

In [4]:

# --- 1. Data Preparation ---
def load_and_chunk_documents(book_path: str):
    """Loads a PDF document and splits it into chunks."""
    try:
        loader = PyPDFLoader(book_path)
        documents = loader.load()
        print(f"Loaded {len(documents)} pages from '{os.path.basename(book_path)}'.")
    except Exception as e:
        print(f"Error loading PDF: {e}")
        print("Please ensure the book path is correct and the PDF is not corrupted.")
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=300,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split book into {len(chunks)} chunks.")
    return chunks


In [5]:

# --- 2. Create Embeddings and Vector Store ---
def create_vector_store(chunks, embedding_model_name: str, faiss_path: str):
    """Creates or loads a FAISS vector store."""
    embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)

    if os.path.exists(faiss_path):
        print(f"Loading existing vector store from '{faiss_path}'...")
        vectorstore = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
        print("Vector store loaded.")
    else:
        print("Creating new vector store (this might take a moment)...")
        vectorstore = FAISS.from_documents(chunks, embeddings)
        vectorstore.save_local(faiss_path)
        print("Vector store created and saved.")
    return vectorstore


In [6]:

# --- 3. Implement RAG ---
def setup_rag_chain(llm_model_name: str, retriever):
    """Sets up the RAG chain with the specified LLM and retriever."""
    try:
        llm = Ollama(model=llm_model_name)
    except Exception as e:
        print(f"Error initializing Ollama with model '{llm_model_name}': {e}")
        print("Please ensure the Ollama server is running and the model is pulled.")
        return None

    rag_prompt_template = """Use the following context to answer the question at the end.
If you don't know the answer, state that you don't know, and do not make up an answer.

Context:
{context}

Question: {question}

Answer:"""
    RAG_PROMPT = PromptTemplate(
        template=rag_prompt_template,
        input_variables=["context", "question"]
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True, # Set to True to get the retrieved chunks
        chain_type_kwargs={"prompt": RAG_PROMPT}
    )
    return qa_chain


In [ ]:

# --- Main Chatbot Logic ---
def main():
    print("Initializing Chatbot...")

    # Load and chunk documents
    chunks = load_and_chunk_documents(BOOK_PATH)
    if not chunks:
        print("Exiting due to document loading error.")
        return

    # Create/Load vector store
    vectorstore = create_vector_store(chunks, EMBEDDING_MODEL_NAME, FAISS_INDEX_PATH)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4}) # Retrieve top 4 relevant chunks

    # Set up RAG chain
    qa_chain = setup_rag_chain(OLLAMA_MODEL_NAME, retriever)
    if not qa_chain:
        print("Exiting due to RAG chain setup error.")
        return

    print(f"\nChatbot initialized with '{OLLAMA_MODEL_NAME}'! Ask questions about your technical book.")
    print("Type 'exit' to quit.")

    while True:
        user_query = input("\nYour Question: ")
        if user_query.lower() == 'exit':
            print("Goodbye!")
            break

        print("Chatbot thinking...")
        try:
            result = qa_chain.invoke({"query": user_query})
            chatbot_response = result["result"]
            source_documents = result.get("source_documents", [])

            print("\n--- Chatbot Response ---")
            print(chatbot_response)

            if source_documents:
                print("\n--- Sources Used (Top Retrieved Chunks) ---")
                for i, doc in enumerate(source_documents):
                    # Attempt to get page number if available in metadata
                    page_info = f"Page: {doc.metadata.get('page') + 1}, " if 'page' in doc.metadata else ""
                    source_filename = os.path.basename(doc.metadata.get('source', 'Unknown'))
                    print(f"Chunk {i+1} ({page_info}Source: {source_filename}):")
                    print(f"  {doc.page_content[:200]}...") # Display first 200 chars of the chunk
            print("------------------------")

        except Exception as e:
            print(f"An error occurred during response generation: {e}")
            print("Please check your Ollama server and model, and ensure your prompt is well-formed.")

if __name__ == "__main__":
    main()

Initializing Chatbot...
Loaded 1059 pages from 'Encyclopedia of Machine Learning.pdf'.
Split book into 2912 chunks.


C:\Users\amith\AppData\Local\Temp\ipykernel_14852\3511803016.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name=embedding_model_name)
c:\Users\amith\anaconda3\envs\torch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\amith\anaconda3\envs\torch_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but yo